In [1]:
import sys
from termcolor import cprint
import numpy as np
import random

In [2]:
class InfoNode():
    def __init__(self, state, parent, action, distance_from):
        self.state = state
        self.parent = parent
        self.action = action
        if parent is None:
            self.distance_to = 0
        else:
            self.distance_to = self.parent.distance_to + 1
        self.distance_from = distance_from

In [3]:
class AStarFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node: InfoNode):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    
    def empty(self):
        return len(self.frontier) == 0

    def size(self):
        return len(self.frontier)
        
    def pop(self):
        if self.empty():
            raise Exception("fontier is empty")
        else:
            minDist = 99999
            for i, node in enumerate(self.frontier):
                if node.distance_from + node.distance_to < minDist:
                    retIndex = i
                    minDist = node.distance_from + node.distance_to
            return self.frontier.pop(retIndex)
    
class GreedyFrontier():
    def __init__(self):
        self.frontier = []
    def add(self, node: InfoNode):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    
    def empty(self):
        return len(self.frontier) == 0

    def size(self):
        return len(self.frontier)
        
    def pop(self):
        if self.empty():
            raise Exception("fontier is empty")
        else:
            minDist = sys.maxsize
            for i, node in enumerate(self.frontier):
                if node.distance_from < minDist:
                    retIndex = i
                    minDist = node.distance_from
            return self.frontier.pop(retIndex)
    


# Knowledge based mazes
# Switch between Greedy and AStar Frontiers to change search algorithm

In [4]:
class InfoMaze():
    def __init__(self, filename:str):
        # Read file
        with open(filename, 'r') as f:
            contents = f.read()
        
        if contents.count('A') != 1:
            raise Exception("Maze needs a single startpoint")
        if contents.count('B') != 1:
            raise Exception("Maze needs a single endpoint")

        contents = contents.splitlines()
        # Record height and width of maze
        self.height = len(contents)
        self.width = max(len(line) for line in contents)

        # Add context
        self.walls = [] # 2D bool array
        for r in range(self.height):
            row = []
            for c in range(self.width):
                # Try block as not all mazes will be rectangular
                try:
                    if contents[r][c] == '#':
                        row.append(True)
                    elif contents[r][c] == ' ':
                        row.append(False)
                    elif contents[r][c] == 'A':
                        self.start = (r,c)
                        row.append(False)
                    elif contents[r][c] == 'B':
                        self.goal = (r,c)
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(True)
            self.walls.append(row)
        self.solution = None

    def get_distance(self, state1:tuple, state2:tuple):
        return abs(state1[0] - state2[0]) + abs(state1[1] - state2[1])


In [5]:
class Agent():
    def __init__(self, frontier):
        self.current_state = None
        self.explored = set()
        self.frontier = frontier
    
    def neighbors(self, maze:InfoMaze):
        row, col = self.current_state
        # All possible actions
        candidates = [
            ('up', (row-1, col)),
            ('down', (row+1, col)),
            ('left', (row, col-1)),
            ('right', (row, col+1)),
        ]
        # Validate actions
        possible = []
        for action, (row, col) in candidates:
            try:
                if not maze.walls[row][col]:
                    possible.append((action, (row, col)))
            except IndexError:
                continue
        return possible

    def solve(self, maze:InfoMaze):
        # Keep track of how many states have been explored (performance measure)
        # self.num_explored = 0

        # Initialize frontier to the starting position
        start = InfoNode(state=maze.start, parent=None, action=None, distance_from=maze.get_distance(maze.start, maze.goal))
        self.frontier.add(start)


        # Search for solution
        while True:
            if self.frontier.empty():
                cprint('No solution', 'red', end='\n\n')
                break
            node = self.frontier.pop()
            self.current_state = node.state
            if self.current_state == maze.goal:
                # Add to explored
                self.explored.add(self.current_state)
                cprint('Solved!', 'green', end='\n\n')
                actions = []
                cells = []

                # Follow breadcrumbs to remake solution path
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                # B -> A --> A -> B
                actions.reverse()
                cells.reverse()
                maze.solution = (actions, cells)
                return
            # Add to explored
            self.explored.add(self.current_state)

            # Add neighbors to frontier
            for action, state in self.neighbors(maze):
                if not self.frontier.contains_state(state):
                    if state not in self.explored: 
                        child = InfoNode(state=state, parent=node, action=action, distance_from=maze.get_distance(state, maze.goal))
                        self.frontier.add(child)
            
    def print_solution(self, maze:InfoMaze, fog_of_war:bool = False):
        for row in range(maze.height):
            for col in range(maze.width):
                if fog_of_war:
                    if (row,col) in self.explored:
                        if (row,col) == maze.start:
                            cprint('A', 'cyan', end='')
                        elif (row,col) == maze.goal:
                            cprint('B', 'cyan', end='')
                        elif maze.walls[row][col] == True:
                            print('#', end='')
                        elif maze.solution is not None and (row,col) in maze.solution[1]: ##########
                            cprint('*', 'green', end='')
                        elif (row,col) in self.explored:
                            cprint('e', 'red', end='')
                        else:
                            print(' ', end='')
                    else:
                        print('?', end='')
                else:
                    if (row,col) == maze.start:
                        cprint('A', 'cyan', end='')
                    elif (row,col) == maze.goal:
                        cprint('B', 'cyan', end='')
                    elif maze.walls[row][col] == True:
                        print('#', end='')
                    elif maze.solution is not None and (row,col) in maze.solution[1]: ##########
                        cprint('*', 'green', end='')
                    elif (row,col) in self.explored:
                        cprint('e', 'red', end='')
                    else:
                        print(' ', end='')
            print()
        print()

        actions, cells = maze.solution
        print('States explored: ' + str(len(self.explored)))
        print('Length of solution: ' + str(len(actions)))



In [8]:
mlazer2 = InfoMaze('maze3.txt')
rational = Agent(GreedyFrontier())
rational.solve(mlazer2)
rational.print_solution(mlazer2, fog_of_war=False)

Solved!

#########################################################################
#                       #                    #      #           #########
#  ##### ##      ##          #########       #      #           #########
## # ###  #  # #      #### ###############   #      #        #  #########
#  # ####    #    #            ######      ###                #  ########
##    ##  #    #####    ################  ####                 #  #######
####      # #     #  # eeee###eeeeeeeeee     ###################  #######
#     ##### #      #  ee#eeeeeeeeeeeeeeee                               #
#            #       eeeeeee#eee##########   ###########   ##############
#            ##########eeeeeeeeeee######eee  #        #   ###############
#  ### # ##       #eee***ee##eeeeeee#eeeeeee #       #   ################
## #   #  #  #    #ee**#***ee#eeeeeeeeeeeeeee#      #   #################
#  # ####    #   eeee*####*e##########eeeeeee#     #   ##################
##    ##  #*** #####e*#eee*ee

1953